In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

from tensorflow import keras


layers = keras.layers
regularizers = keras.regularizers

#HYPERPARAMETERS
BATCH_SIZE = 32
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [8]:
class_names = [
    "Airplane",
    "Autmobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

In [5]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# print(ds_info)

In [6]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


def augment(image, label):
    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_flip_left_right(image)

    return image, label

In [7]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

In [9]:
def get_model():
    model = keras.Sequential(
        [
            layers.Input((32, 32, 3)),
            layers.Conv2D(8, 3, padding="same", activation="relu"),
            layers.Conv2D(16, 3, padding="same", activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dropout(0.1),
            layers.Dense(10),
        ]
    )

    return model

In [11]:
model = get_model()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [12]:
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir="tb_callback_dir", histogram_freq=1,
)

In [13]:
model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
    callbacks=[tensorboard_callback],
    verbose=2,
)

Epoch 1/5
1563/1563 - 60s - loss: 1.5417 - accuracy: 0.4450 - val_loss: 1.2507 - val_accuracy: 0.5599 - 60s/epoch - 38ms/step
Epoch 2/5
1563/1563 - 54s - loss: 1.2342 - accuracy: 0.5632 - val_loss: 1.0772 - val_accuracy: 0.6256 - 54s/epoch - 35ms/step
Epoch 3/5
1563/1563 - 52s - loss: 1.1231 - accuracy: 0.6022 - val_loss: 1.0106 - val_accuracy: 0.6485 - 52s/epoch - 33ms/step
Epoch 4/5
1563/1563 - 51s - loss: 1.0588 - accuracy: 0.6271 - val_loss: 0.9495 - val_accuracy: 0.6679 - 51s/epoch - 33ms/step
Epoch 5/5
1563/1563 - 54s - loss: 1.0168 - accuracy: 0.6406 - val_loss: 0.9228 - val_accuracy: 0.6782 - 54s/epoch - 34ms/step
